# optimization algorithm using CNN for MNIST 
- CNN model is compiled using Adagrad optimization


In [40]:
from keras.datasets import mnist 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization 
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical 

In [41]:
batch_size=250
epochs=5
no_classes=10
validation_split=0.2
verbose=1

#load KMNIST dataset
(xtrain,ytrain),(xtest,ytest)=mnist.load_data()
# print(xtrain)
xtrain.shape, xtest.shape


((60000, 28, 28), (10000, 28, 28))

In [42]:
# convert target variables to multiclass classification
ytrain_norm=to_categorical(ytrain)
ytrain_norm
ytest_norm=to_categorical(ytest)
ytest_norm

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [43]:
#keras input shape
input_shape=(xtrain.shape[1],xtrain.shape[2],1)
#reshape
xtrain=xtrain.reshape(xtrain.shape[0],xtrain.shape[1],xtrain.shape[2],1)
xtest=xtest.reshape(xtest.shape[0],xtest.shape[1],xtest.shape[2],1)
xtrain=xtrain.astype('float32')
xtest=xtest.astype('float32')

In [44]:
#normalize the data
xtrain=xtrain/255.
xtest=xtest/255.


# creating the model

In [45]:
model=Sequential([
    Conv2D(32,kernel_size=(3,3),activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    Conv2D(64,kernel_size=(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    Flatten(),
    Dense(256,activation='relu'),
    BatchNormalization(),
    Dense(10,activation='softmax')
])



In [66]:
# Adagrad optimizer
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.1,
    epsilon=1e-07
),metrics=['accuracy'])

In [51]:
# Adadelta optimizer
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adadelta(
    learning_rate=0.001,
    rho=0.95,
    epsilon=1e-07,
    name='Adadelta'
))

In [54]:
# Adam-adaptive optimizer
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(
    learning_rate=0.001), metrics=['accuracy'])

In [61]:
#adabound optimizer
from keras_adabound import AdaBound
model.compile(loss='categorical_crossentropy',optimizer=AdaBound(
    lr=1e-3, final_lr=0.1
))

ModuleNotFoundError: No module named 'keras_adabound'

In [67]:
# fit data to model
history = model.fit(xtrain,ytrain_norm,batch_size=250,epochs=5,verbose=1,validation_split=0.2)

Epoch 1/5
192/192 [==============================] - 2s 8ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.0305 - val_accuracy: 0.9916
Epoch 2/5
192/192 [==============================] - 1s 6ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0305 - val_accuracy: 0.9917
Epoch 3/5
192/192 [==============================] - 1s 7ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0305 - val_accuracy: 0.9918
Epoch 4/5
192/192 [==============================] - 1s 6ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0303 - val_accuracy: 0.9918
Epoch 5/5
192/192 [==============================] - 1s 6ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.0303 - val_accuracy: 0.9918


# generate generalization metrics

In [56]:
score = model.evaluate(xtest,ytest_norm,verbose=0)
score
# print(f'Test loss using Adagrad: {score[0]} / Test accuracy: {score[1]}')

[0.03495953232049942, 0.9898999929428101]